In [1]:
%matplotlib inline
from keras.models import Model
from keras.layers import *
from keras.optimizers import *
from keras.utils import plot_model

import keras.initializers as kinit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dqnroute.utils import *

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = pd.read_csv('logs/data_generated2.csv', names=get_data_cols(10))

In [3]:
import gc
gc.collect()

0

In [4]:
data.head()

,time,pkg_id,cur_node,dst_0,dst_1,dst_2,dst_3,dst_4,dst_5,dst_6,...,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,10.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-10.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-30.0,-1000000.0
1,20.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1000000.0,-20.0,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
2,20.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0,-1000000.0
3,30.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-1000000.0,-10.0,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
4,40.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-20.0,-1000000.0


In [18]:
target_cols = get_target_cols(10)
neighbors_cols = get_neighbors_cols(10)
addr_cols = get_addr_cols(10)
dst_cols = get_dst_cols(10)
amatrix_cols = get_amatrix_cols(10)

targets = data[target_cols]
neighbors_inp = data[neighbors_cols]
addr_inp = data[addr_cols]
dst_inp = data[dst_cols]
amatrix_inp = data[amatrix_cols]

In [19]:
neighbors_inp.head()

,neighbors_0,neighbors_1,neighbors_2,neighbors_3,neighbors_4,neighbors_5,neighbors_6,neighbors_7,neighbors_8,neighbors_9
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [20]:
targets.head()

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,-10.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-30.0,-1000000.0
1,-1000000.0,-20.0,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
2,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0,-1000000.0
3,-1000000.0,-10.0,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
4,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-20.0,-1000000.0


In [22]:
neighbors_input_dim = neighbors_inp.shape[1]
addr_input_dim = addr_inp.shape[1]
dst_input_dim = dst_inp.shape[1]
amatrix_input_dim = amatrix_inp.shape[1]
output_dim = targets.shape[1]

In [23]:
neighbors_input = Input(shape=(neighbors_input_dim,), name='neighbors_input')
addr_input = Input(shape=(addr_input_dim,), name='addr_input')
dst_input = Input(shape=(dst_input_dim,), name='dst_input')
amatrix_input = Input(shape=(amatrix_input_dim,), name='amatrix_input')

x = concatenate([neighbors_input, addr_input, dst_input, amatrix_input])
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(output_dim, activation='linear')(x)

y = Lambda(lambda x: (1 - x) * -1000000)(neighbors_input)

x = add([x, y])
model = Model(inputs=[neighbors_input, addr_input, dst_input, amatrix_input], outputs=x)

model.compile(loss='mse', optimizer='rmsprop')

In [24]:
N_train, N_test, A_train, A_test, D_train, D_test, M_train, M_test, y_train, y_test = \
    train_test_split(neighbors_inp.values,
                     addr_inp.values,
                     dst_inp.values,
                     amatrix_inp.values,
                     targets.values,
                     test_size=0.2, random_state=42)

In [27]:
model.fit([N_train, A_train, D_train, M_train], y_train, epochs=30)

Epoch 1/30
184857/184857 [==============================] - 10s - loss: 0.4483    
Epoch 2/30
184857/184857 [==============================] - 11s - loss: 0.4350    
Epoch 3/30
184857/184857 [==============================] - 12s - loss: 0.4229    
Epoch 4/30
184857/184857 [==============================] - 12s - loss: 0.4126    
Epoch 5/30
184857/184857 [==============================] - 11s - loss: 0.4016    
Epoch 6/30
184857/184857 [==============================] - 10s - loss: 0.3930    
Epoch 7/30
184857/184857 [==============================] - 10s - loss: 0.3837    
Epoch 8/30
184857/184857 [==============================] - 10s - loss: 0.3736    
Epoch 9/30
184857/184857 [==============================] - 10s - loss: 0.3655    
Epoch 10/30
184857/184857 [==============================] - 10s - loss: 0.3573    
Epoch 11/30
184857/184857 [==============================] - 12s - loss: 0.3489    
Epoch 12/30
184857/184857 [==============================] - 11s - loss: 0.3404    
E

In [28]:
model.evaluate([N_test, A_test, D_test, M_test], y_test)

44800/46215 [============================>.] - ETA: 0s

0.22675469348753838

In [30]:
model.predict(list(map(lambda x: x.reshape((1, 10)), [N_test[0], A_test[0], D_test[0]])) + [M_test[0].reshape((1, 100))])

array([[ -1.00000000e+06,  -1.00000019e+06,  -2.00017242e+01,
         -9.99999500e+05,  -1.00000050e+06,  -1.00000031e+06,
         -1.00000025e+06,  -9.99999375e+05,  -9.99999812e+05,
         -9.99999688e+05]], dtype=float32)

In [31]:
y_test[0]

array([ -1.00000000e+06,  -1.00000000e+06,  -2.00000000e+01,
        -1.00000000e+06,  -1.00000000e+06,  -1.00000000e+06,
        -1.00000000e+06,  -1.00000000e+06,  -1.00000000e+06,
        -1.00000000e+06])

In [32]:
model.save('router_network.h5')

In [ ]:
plot_model(model, show_shapes=True)